# Synthetic Financial Datasets For Fraud Detection

Kaggle赛题，上传者用“PaySim”——一个综合数据集生成的模拟器，从隐私聚合数据生成一个合成数据集，模拟了移动支付的正常操作，并添加了恶意行为来实施关于支付日志是否属于欺诈行为（即二分类问题，欺诈/正常交易）的预测，评估欺诈检测方法的性能。

In [83]:
import pandas as pd
import numpy as np

In [84]:
df = pd.read_csv('/Users/x/Downloads/PS_20174392719_1491204439457_log.csv')
df = df.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', \
                        'oldbalanceDest':'oldBalanceDest', 'newbalanceDest':'newBalanceDest'})
df.head()

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [85]:
df.shape

(6362620, 11)

In [86]:
df.isnull().values.any()

False

In [87]:
dfRel = df.loc[:,['nameOrig','nameDest']]
print(dfRel.head())

nameOrig     nameDest
0  C1231006815  M1979787155
1  C1666544295  M2044282225
2  C1305486145   C553264065
3   C840083671    C38997010
4  C2048537720  M1230701703


In [88]:
print('\nRepeats of name in nameOrig is \n{}'.format(dfRel.nameOrig.duplicated(keep=False).value_counts()))
print('\nRepeats of name in nameDest is \n{}'.format(dfRel.nameDest.duplicated(keep=False).value_counts()))


Repeats of name in nameOrig is 
False    6344009
True       18611
Name: nameOrig, dtype: int64

Repeats of name in nameDest is 
True     4099916
False    2262704
Name: nameDest, dtype: int64


In [89]:
repeatedOrig = dfRel[dfRel.nameOrig.duplicated(keep=False)]
repeatedDest = dfRel[dfRel.nameDest.duplicated(keep=False)]
print(repeatedOrig)
print(repeatedDest)

nameOrig     nameDest
95        C835773569  C1359044626
834      C1101598632   C248609774
835      C1032568028   C667346055
1241     C1892869131    C33524623
2165     C1859823482  C1529115495
...              ...          ...
6359886   C876817986  M2021251796
6360204  C1313642079   C927206109
6360821   C188194996  C1210630308
6361231  C1170262181  C1991152940
6362423   C850961884  C1203132980

[18611 rows x 2 columns]
            nameOrig     nameDest
2        C1305486145   C553264065
3         C840083671    C38997010
9         C712410124   C195600860
10       C1900366749   C997608398
15        C905080434   C476402209
...              ...          ...
6362611   C994950684  C1662241365
6362613  C1436118706  C1240760502
6362615   C786484425   C776919290
6362617  C1162922333  C1365125890
6362619  C1280323807   C873221189

[4099916 rows x 2 columns]


In [90]:
repeatedName = pd.concat([repeatedOrig,repeatedDest])

In [91]:
print(repeatedName)
print(repeatedName.index.is_unique) 
# repeatedNameDup = repeatedName[repeatedName.duplicated(keep=False)]
# print(repeatedNameDup)
repeatedName_uniq = repeatedName.drop_duplicates()
print(repeatedName_uniq)
# print(repeatedName_uniq.index.is_unique) 
repeatedName_uniqSorted = repeatedName_uniq.sort_index()
print(repeatedName_uniqSorted)

nameOrig     nameDest
95        C835773569  C1359044626
834      C1101598632   C248609774
835      C1032568028   C667346055
1241     C1892869131    C33524623
2165     C1859823482  C1529115495
...              ...          ...
6362611   C994950684  C1662241365
6362613  C1436118706  C1240760502
6362615   C786484425   C776919290
6362617  C1162922333  C1365125890
6362619  C1280323807   C873221189

[4118527 rows x 2 columns]
False
            nameOrig     nameDest
95        C835773569  C1359044626
834      C1101598632   C248609774
835      C1032568028   C667346055
1241     C1892869131    C33524623
2165     C1859823482  C1529115495
...              ...          ...
6362611   C994950684  C1662241365
6362613  C1436118706  C1240760502
6362615   C786484425   C776919290
6362617  C1162922333  C1365125890
6362619  C1280323807   C873221189

[4106555 rows x 2 columns]
            nameOrig     nameDest
2        C1305486145   C553264065
3         C840083671    C38997010
9         C712410124   C19560086

In [92]:
dfRel.to_csv('/Users/x/payRel.csv')

In [93]:
print('\n The types of fraudulent transactions are {}'.format(\
list(df.loc[df.isFraud == 1].type.drop_duplicates().values))) # only 'CASH_OUT' & 'TRANSFER'

dfFraudTransfer = df.loc[(df.isFraud == 1) & (df.type == 'TRANSFER')]
dfFraudCashout = df.loc[(df.isFraud == 1) & (df.type == 'CASH_OUT')]

print ('\n The number of fraudulent TRANSFERs = {}'.\
       format(len(dfFraudTransfer))) # 4097

print ('\n The number of fraudulent CASH_OUTs = {}'.\
       format(len(dfFraudCashout))) # 4116


 The types of fraudulent transactions are ['TRANSFER', 'CASH_OUT']

 The number of fraudulent TRANSFERs = 4097

 The number of fraudulent CASH_OUTs = 4116


In [94]:
print('\nThe type of transactions in which isFlaggedFraud is set: \
{}'.format(list(df.loc[df.isFlaggedFraud == 1].type.drop_duplicates()))) 
                                                            # only 'TRANSFER'

dfTransfer = df.loc[df.type == 'TRANSFER']
dfFlagged = df.loc[df.isFlaggedFraud == 1]
dfNotFlagged = df.loc[df.isFlaggedFraud == 0]

print('\nMin amount transacted when isFlaggedFraud is set= {}'\
                                  .format(dfFlagged.amount.min())) # 353874.22

print('\nMax amount transacted in a TRANSFER where isFlaggedFraud is not set=\
 {}'.format(dfTransfer.loc[dfTransfer.isFlaggedFraud == 0].amount.max())) # 92445516.64


The type of transactions in which isFlaggedFraud is set: ['TRANSFER']

Min amount transacted when isFlaggedFraud is set= 353874.22

Max amount transacted in a TRANSFER where isFlaggedFraud is not set= 92445516.64


In [95]:
print('\nThe number of TRANSFERs where isFlaggedFraud = 0, yet oldBalanceDest = 0 and\
newBalanceDest = 0: {}'.\
format(len(dfTransfer.loc[(dfTransfer.isFlaggedFraud == 0) & \
(dfTransfer.oldBalanceDest == 0) & (dfTransfer.newBalanceDest == 0)]))) # 4158


The number of TRANSFERs where isFlaggedFraud = 0, yet oldBalanceDest = 0 andnewBalanceDest = 0: 4158


In [96]:
print('\nMin, Max of oldBalanceOrig for isFlaggedFraud = 1 TRANSFERs: {}'.\
format([round(dfFlagged.oldBalanceOrig.min()), round(dfFlagged.oldBalanceOrig.max())]))

print('\nMin, Max of oldBalanceOrig for isFlaggedFraud = 0 TRANSFERs where \
oldBalanceOrig = \
newBalanceOrig: {}'.format(\
[dfTransfer.loc[(dfTransfer.isFlaggedFraud == 0) & (dfTransfer.oldBalanceOrig \
== dfTransfer.newBalanceOrig)].oldBalanceOrig.min(), \
round(dfTransfer.loc[(dfTransfer.isFlaggedFraud == 0) & (dfTransfer.oldBalanceOrig \
               == dfTransfer.newBalanceOrig)].oldBalanceOrig.max())]))


Min, Max of oldBalanceOrig for isFlaggedFraud = 1 TRANSFERs: [353874.0, 19585040.0]

Min, Max of oldBalanceOrig for isFlaggedFraud = 0 TRANSFERs where oldBalanceOrig = newBalanceOrig: [0.0, 575668.0]


In [97]:
print('\nHave originators of transactions flagged as fraud transacted more than \
once? {}'\
.format((dfFlagged.nameOrig.isin(pd.concat([dfNotFlagged.nameOrig, \
                                    dfNotFlagged.nameDest]))).any())) # False

print('\nHave destinations for transactions flagged as fraud initiated\
 other transactions? \
{}'.format((dfFlagged.nameDest.isin(dfNotFlagged.nameOrig)).any())) # False

# Since only 2 destination accounts of 16 that have 'isFlaggedFraud' set have been
# destination accounts more than once,
# clearly 'isFlaggedFraud' being set is independent of whether a 
# destination account has been used before or not

print('\nHow many destination accounts of transactions flagged as fraud have been \
destination accounts more than once?: {}'\
.format(sum(dfFlagged.nameDest.isin(dfNotFlagged.nameDest)))) # 2


Have originators of transactions flagged as fraud transacted more than once? False

Have destinations for transactions flagged as fraud initiated other transactions? False

How many destination accounts of transactions flagged as fraud have been destination accounts more than once?: 2


In [98]:
print('\nAre there any merchants among originator accounts for CASH_IN transactions? {}'.format((df.loc[df.type == 'CASH_IN'].nameOrig.str.contains('M')).any())) # False
print('\nAre there any merchants among destination accounts for CASH_OUT transactions? {}'.format((df.loc[df.type == 'CASH_OUT'].nameDest.str.contains('M')).any())) # False
print('\nAre there merchants among any originator accounts? {}'.format(df.nameOrig.str.contains('M').any())) # False
print('\nAre there any transactions having merchants among destination accounts other than the PAYMENT type? {}'.format((df.loc[df.nameDest.str.contains('M')].type != 'PAYMENT').any())) # False


Are there any merchants among originator accounts for CASH_IN transactions? False

Are there any merchants among destination accounts for CASH_OUT transactions? False

Are there merchants among any originator accounts? False

Are there any transactions having merchants among destination accounts other than the PAYMENT type? False


In [99]:
print('\nWithin fraudulent transactions, are there destinations for TRANSFERS that are also originators for CASH_OUTs? {}'.format((dfFraudTransfer.nameDest.isin(dfFraudCashout.nameOrig)).any())) # False


Within fraudulent transactions, are there destinations for TRANSFERS that are also originators for CASH_OUTs? False


In [100]:
dfNotFraud = df.loc[df.isFraud == 0]

In [101]:
print('\nFraudulent TRANSFERs whose destination accounts are originators of genuine CASH_OUTs:')
dfFraudTransfer.loc[dfFraudTransfer.nameDest.isin(dfNotFraud.loc[dfNotFraud.type == 'CASH_OUT'].nameOrig.drop_duplicates())]


Fraudulent TRANSFERs whose destination accounts are originators of genuine CASH_OUTs:


,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud
1030443,65,TRANSFER,1282971.57,C1175896731,1282971.57,0.0,C1714931087,0.0,0.0,1,0
6039814,486,TRANSFER,214793.32,C2140495649,214793.32,0.0,C423543548,0.0,0.0,1,0
6362556,738,TRANSFER,814689.88,C2029041842,814689.88,0.0,C1023330867,0.0,0.0,1,0


In [102]:
print('\nFraudulent TRANSFER to C423543548 occured at step = 486 whereas genuine CASH_OUT from this account occured earlier at step = {}'.format(dfNotFraud.loc[(dfNotFraud.type == 'CASH_OUT') & (dfNotFraud.nameOrig == 'C423543548')].step.values)) # 185


Fraudulent TRANSFER to C423543548 occured at step = 486 whereas genuine CASH_OUT from this account occured earlier at step = [185]


In [103]:
X = df.loc[(df.type == 'TRANSFER') | (df.type == 'CASH_OUT')]
Y = X['isFraud']

# Eliminate columns shown to be irrelevant for analysis in the EDA
X.drop(['isFraud','nameOrig', 'nameDest', 'isFlaggedFraud'], axis = 1)
X

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud
2,1,TRANSFER,181.00,C1305486145,181.00,0.0,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.0,C38997010,21182.00,0.00,1,0
15,1,CASH_OUT,229133.94,C905080434,15325.00,0.0,C476402209,5083.00,51513.44,0,0
19,1,TRANSFER,215310.30,C1670993182,705.00,0.0,C1100439041,22425.00,0.00,0,0
24,1,TRANSFER,311685.89,C1984094095,10835.00,0.0,C932583850,6267.00,2719172.89,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.0,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.0,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.0,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.0,C2080388513,0.00,0.00,1,0


In [104]:
# Binary-encoding of labelled data in 'type'
X = X.replace(['TRANSFER','CASH_OUT'],[0,1])
# X.dtypes

In [105]:
Xfraud = X.loc[Y == 1]
XnonFraud = X.loc[Y == 0]

print('\nThe fraction of fraudulent transactions with \'oldBalanceDest\' = \'newBalanceDest\' = 0 \nalthough the transacted \'amount\' is non-zero is: {}%'.\
format(len(Xfraud.loc[(Xfraud.oldBalanceDest == 0) & (Xfraud.newBalanceDest == 0) & (Xfraud.amount)]) / len(Xfraud)*100))

print('\nThe fraction of genuine transactions with \'oldBalanceDest\' = \'newBalanceDest\' = 0 \nalthough the transacted \'amount\' is non-zero is: {}%'.\
format(len(XnonFraud.loc[(XnonFraud.oldBalanceDest == 0) & (XnonFraud.newBalanceDest == 0) & (XnonFraud.amount)]) / len(XnonFraud)*100))


The fraction of fraudulent transactions with 'oldBalanceDest' = 'newBalanceDest' = 0 
although the transacted 'amount' is non-zero is: 49.55558261293072%

The fraction of genuine transactions with 'oldBalanceDest' = 'newBalanceDest' = 0 
although the transacted 'amount' is non-zero is: 0.061762452773083454%


In [106]:
X.loc[(X.oldBalanceDest == 0) & (X.newBalanceDest == 0) & (X.amount != 0), ['oldBalanceDest', 'newBalanceDest']] = - 1
X

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud
2,1,0,181.00,C1305486145,181.00,0.0,C553264065,-1.00,-1.00,1,0
3,1,1,181.00,C840083671,181.00,0.0,C38997010,21182.00,0.00,1,0
15,1,1,229133.94,C905080434,15325.00,0.0,C476402209,5083.00,51513.44,0,0
19,1,0,215310.30,C1670993182,705.00,0.0,C1100439041,22425.00,0.00,0,0
24,1,0,311685.89,C1984094095,10835.00,0.0,C932583850,6267.00,2719172.89,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,1,339682.13,C786484425,339682.13,0.0,C776919290,0.00,339682.13,1,0
6362616,743,0,6311409.28,C1529008245,6311409.28,0.0,C1881841831,-1.00,-1.00,1,0
6362617,743,1,6311409.28,C1162922333,6311409.28,0.0,C1365125890,68488.84,6379898.11,1,0
6362618,743,0,850002.52,C1685995037,850002.52,0.0,C2080388513,-1.00,-1.00,1,0


In [107]:
print('Number of -1 values in oldBalanceDest is {}'.format(X.groupby(['oldBalanceDest']).size()[-1]))
print('Number of -1 values in oldBalanceDest is {}'.format(X.groupby(['oldBalanceDest']).size()[-1]))

Number of -1 values in oldBalanceDest is 5776
Number of -1 values in oldBalanceDest is 5776


In [108]:
print('\nThe fraction of fraudulent transactions with \'oldBalanceOrig\' = \'newBalanceOrig\' = 0 \nalthough the transacted \'amount\' is non-zero is: {}%'.\
format(len(Xfraud.loc[(Xfraud.oldBalanceOrig == 0) & (Xfraud.newBalanceOrig == 0) & (Xfraud.amount)]) / len(Xfraud)*100))

print('\nThe fraction of genuine transactions with \'oldBalanceOrig\' = \'newBalanceOrig\' = 0 \nalthough the transacted \'amount\' is non-zero is: {}%'.\
format(len(XnonFraud.loc[(XnonFraud.oldBalanceOrig == 0) & (XnonFraud.newBalanceOrig == 0) & (XnonFraud.amount)]) / len(XnonFraud)*100))


The fraction of fraudulent transactions with 'oldBalanceOrig' = 'newBalanceOrig' = 0 
although the transacted 'amount' is non-zero is: 0.30439547059539757%

The fraction of genuine transactions with 'oldBalanceOrig' = 'newBalanceOrig' = 0 
although the transacted 'amount' is non-zero is: 47.37321319703598%


In [109]:
X.loc[(X.oldBalanceOrig == 0) & (X.newBalanceOrig == 0) & (X.amount != 0), ['oldBalanceOrig', 'newBalanceOrig']] = np.nan
X

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud
2,1,0,181.00,C1305486145,181.00,0.0,C553264065,-1.00,-1.00,1,0
3,1,1,181.00,C840083671,181.00,0.0,C38997010,21182.00,0.00,1,0
15,1,1,229133.94,C905080434,15325.00,0.0,C476402209,5083.00,51513.44,0,0
19,1,0,215310.30,C1670993182,705.00,0.0,C1100439041,22425.00,0.00,0,0
24,1,0,311685.89,C1984094095,10835.00,0.0,C932583850,6267.00,2719172.89,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,1,339682.13,C786484425,339682.13,0.0,C776919290,0.00,339682.13,1,0
6362616,743,0,6311409.28,C1529008245,6311409.28,0.0,C1881841831,-1.00,-1.00,1,0
6362617,743,1,6311409.28,C1162922333,6311409.28,0.0,C1365125890,68488.84,6379898.11,1,0
6362618,743,0,850002.52,C1685995037,850002.52,0.0,C2080388513,-1.00,-1.00,1,0


In [110]:
print('Number of \'NaN\' values in newBalanceOrig is {}'.format(X.newBalanceOrig.isna().sum()))
print('Number of valid values in newBalanceOrig is {}'.format(X.newBalanceOrig.count()))

Number of 'NaN' values in newBalanceOrig is 1308566
Number of valid values in newBalanceOrig is 1461843
